In [1]:
!pip install -q transformers datasets scikit-learn accelerate torch pandas

import random
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
    set_seed,
)
from datasets import load_dataset
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight


EXPERIMENT_CONFIG = [
    {"model_name": "microsoft/deberta-v3-large", "lr": 6e-6, "batch_size": 4},
    {"model_name": "microsoft/deberta-v3-large", "lr": 1e-5, "batch_size": 4},
    {"model_name": "xlnet-large-cased",          "lr": 1e-5, "batch_size": 4},
    {"model_name": "xlnet-large-cased",          "lr": 2e-5, "batch_size": 4},
]

OUTPUT_DIR_BASE = "./comparison_task2"
MAX_LEN = 512
EPOCHS = 10
GRAD_ACCUMULATION = 4
SEED = 42

set_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

dataset = load_dataset("ailsntua/QEvasion")

def preprocess_base(example):
    clarity = example.get("clarity_label", "Unknown") or "Unknown"
    text = f"Context: {clarity} | Question: {example['question']} Answer: {example['interview_answer']}"
    return {"text": text, "evasion_label": example["evasion_label"]}

full_data = dataset["train"].map(preprocess_base)
full_data = full_data.class_encode_column("evasion_label")

split1 = full_data.train_test_split(test_size=0.1, seed=SEED, stratify_by_column="evasion_label")
train_dev_ds = split1["train"]
held_out_test_ds = split1["test"]

split2 = train_dev_ds.train_test_split(test_size=0.1, seed=SEED, stratify_by_column="evasion_label")
train_ds = split2["train"]
eval_ds = split2["test"]

labels = train_ds.features["evasion_label"].names
label2id = {name: i for i, name in enumerate(labels)}
id2label = {i: name for name, i in label2id.items()}

y_train = train_ds["evasion_label"]
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y_train), y=y_train)
device = "cuda" if torch.cuda.is_available() else "cpu"
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": (preds == labels).mean(),
        "macro_f1": f1_score(labels, preds, average="macro"),
    }

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = nn.CrossEntropyLoss(weight=class_weights_tensor, label_smoothing=0.1)
        loss = loss_fct(logits.view(-1, model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

results_table = []

print(f"{'Model':<30} | {'LR':<10} | {'F1 Score':<10} | {'Accuracy':<10}")
print("-" * 70)

for run in EXPERIMENT_CONFIG:
    model_name = run["model_name"]
    lr = run["lr"]


    tokenizer = AutoTokenizer.from_pretrained(model_name)


    if "xlnet" in model_name:
        tokenizer.padding_side = "left"
    else:
        tokenizer.padding_side = "right"

    def tokenize_fn(batch):
        return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=MAX_LEN)

    current_train = train_ds.map(tokenize_fn, batched=True).map(lambda x: {"labels": x["evasion_label"]})
    current_eval = eval_ds.map(tokenize_fn, batched=True).map(lambda x: {"labels": x["evasion_label"]})
    current_test = held_out_test_ds.map(tokenize_fn, batched=True).map(lambda x: {"labels": x["evasion_label"]})

    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=len(labels),
        id2label=id2label,
        label2id=label2id,
    )

    run_output_dir = f"{OUTPUT_DIR_BASE}/{model_name.split('/')[-1]}_lr{lr}"
    args = TrainingArguments(
        output_dir=run_output_dir,
        learning_rate=lr,
        per_device_train_batch_size=run["batch_size"],
        per_device_eval_batch_size=run["batch_size"] * 2,
        gradient_accumulation_steps=GRAD_ACCUMULATION,
        num_train_epochs=EPOCHS,
        weight_decay=0.01,
        warmup_ratio=0.1,
        fp16=True,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="macro_f1",
        report_to="none",
        save_total_limit=1,
        seed=SEED,
    )

    trainer = WeightedTrainer(
        model=model,
        args=args,
        train_dataset=current_train,
        eval_dataset=current_eval,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    )

    print(f"\nTraining {model_name} with LR={lr}")
    trainer.train()

    eval_results = trainer.evaluate(current_test)
    f1 = eval_results["eval_macro_f1"]
    acc = eval_results["eval_accuracy"]

    results_table.append({
        "Model": model_name,
        "Learning Rate": lr,
        "Macro F1": f1,
        "Accuracy": acc
    })

    print(f"Result: {model_name} (LR: {lr}) -> F1: {f1:.4f}")

df_results = pd.DataFrame(results_table).sort_values(by="Macro F1", ascending=False)
print(df_results)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.90M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/259k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3448 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/311 [00:00<?, ? examples/s]

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-large-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Training xlnet-large-cased with LR=2e-05...


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,No log,2.526704,0.205788,0.037926
2,No log,2.512107,0.305466,0.051998
3,10.307600,2.503928,0.305466,0.051998
4,10.307600,2.517178,0.305466,0.051998
5,10.307600,2.469103,0.260450,0.088138
6,10.190600,2.213534,0.180064,0.093922
7,10.190600,1.933395,0.456592,0.250540
8,10.190600,1.892194,0.463023,0.300091
9,8.157000,1.885592,0.556270,0.295046
10,8.157000,1.884456,0.533762,0.358476


Result: xlnet-large-cased (LR: 2e-05) -> F1: 0.3827
                        Model  Learning Rate  Macro F1  Accuracy
0  microsoft/deberta-v3-large       0.000006  0.570860  0.608696
1  microsoft/deberta-v3-large       0.000010  0.542350  0.594203
2           xlnet-large-cased       0.000010  0.513815  0.591304
3           xlnet-large-cased       0.000020  0.382677  0.576812
